In [5]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMA
import matplotlib as plp
import lightgbm as lgb

In [6]:
df_train = pd.read_csv(
    '../Data/train_set.csv', usecols=[1, 2, 3, 4],
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"]
)

In [7]:
df_test = pd.read_csv(
    "../Data/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [8]:
items = pd.read_csv(
    "../Data/items.csv",
).set_index("item_nbr")

In [9]:
df_train = df_train.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_train.columns = df_train.columns.get_level_values(1)

In [10]:
items = items.reindex(df_train.index.get_level_values(1))

In [11]:
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

In [12]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "mean_3_2017": get_timespan(df_train, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_train, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_train, t2017, 14, 14).mean(axis=1).values,
        "mean_16_2017": get_timespan(df_train, t2017, 16, 16).mean(axis=1).values
    })
    if is_train:
        y = df_train[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

In [13]:
print("Preparing dataset...")
t2017 = date(2017, 5, 16)
X_l, y_l = [], []
for i in range(9):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(t2017 + delta)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

Preparing dataset...


In [14]:
X_val, y_val = prepare_dataset(date(2017, 7, 23))

In [15]:
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

In [17]:
print("Training and predicting models...")
params = {
    'num_leaves': 80,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 16
}

MAX_ROUNDS = 5000
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 9) * 0.25 + 1
    )
    
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        categorical_feature=cate_vars,
        weight=items["perishable"] * 0.25 + 1
    )
    
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=50
    )
    
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

Training and predicting models...
Step 1


/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.464951	valid_1's l2: 0.579554
[100]	training's l2: 0.380001	valid_1's l2: 0.460505
[150]	training's l2: 0.368057	valid_1's l2: 0.436342
[200]	training's l2: 0.366136	valid_1's l2: 0.429946
[250]	training's l2: 0.365604	valid_1's l2: 0.42805
[300]	training's l2: 0.365295	valid_1's l2: 0.427396
[350]	training's l2: 0.365064	valid_1's l2: 0.427158
[400]	training's l2: 0.364848	valid_1's l2: 0.42704
[450]	training's l2: 0.364648	valid_1's l2: 0.427032
[500]	training's l2: 0.364455	valid_1's l2: 0.426979
[550]	training's l2: 0.364261	valid_1's l2: 0.426946
[600]	training's l2: 0.364072	valid_1's l2: 0.426988
Early stopping, best iteration is:
[551]	training's l2: 0.364257	valid_1's l2: 0.426942
mean_14_2017: 8769569.52
mean_7_2017: 8301834.56
mean_16_2017: 1758688.03
mean_3_2017: 1083723.72
Step 2


/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.478981	valid_1's l2: 0.483198
[100]	training's l2: 0.391964	valid_1's l2: 0.394729
[150]	training's l2: 0.379774	valid_1's l2: 0.381855
[200]	training's l2: 0.377792	valid_1's l2: 0.379788
[250]	training's l2: 0.37723	valid_1's l2: 0.379337
[300]	training's l2: 0.376895	valid_1's l2: 0.379195
[350]	training's l2: 0.37664	valid_1's l2: 0.379143
[400]	training's l2: 0.376406	valid_1's l2: 0.379091
[450]	training's l2: 0.376189	valid_1's l2: 0.379078
[500]	training's l2: 0.375986	valid_1's l2: 0.379069
[550]	training's l2: 0.375791	valid_1's l2: 0.379066
Early stopping, best iteration is:
[524]	training's l2: 0.37589	valid_1's l2: 0.379055
mean_14_2017: 12773027.38
mean_7_2017: 5310976.57
mean_16_2017: 2173881.34
mean_3_2017: 200379.92
Step 3


...

Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.543072	valid_1's l2: 0.611643
[100]	training's l2: 0.463885	valid_1's l2: 0.517841
[150]	training's l2: 0.452618	valid_1's l2: 0.500924
[200]	training's l2: 0.450613	valid_1's l2: 0.496807
[250]	training's l2: 0.44999	valid_1's l2: 0.495556
[300]	training's l2: 0.449629	valid_1's l2: 0.495117
[350]	training's l2: 0.449325	valid_1's l2: 0.494891
[400]	training's l2: 0.449055	valid_1's l2: 0.49477
[450]	training's l2: 0.44879	valid_1's l2: 0.494665
[500]	training's l2: 0.448542	valid_1's l2: 0.494612
[550]	training's l2: 0.448309	valid_1's l2: 0.49454
[600]	training's l2: 0.448088	valid_1's l2: 0.494503
[650]	training's l2: 0.447875	valid_1's l2: 0.494454
[700]	training's l2: 0.447676	valid_1's l2: 0.49442
[750]	training's l2: 0.447487	valid_1's l2: 0.494396
Early stopping, best iteration is:
[730]	training's l2: 0.44756	valid_1's l2: 0.494378
mean_16_2017: 15180815.65
mean_14_2017: 2931841.63
mean_7_2017

/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.537159	valid_1's l2: 0.700827
[100]	training's l2: 0.4635	valid_1's l2: 0.596536
[150]	training's l2: 0.45312	valid_1's l2: 0.574926
[200]	training's l2: 0.451327	valid_1's l2: 0.569131
[250]	training's l2: 0.450767	valid_1's l2: 0.567222
[300]	training's l2: 0.450424	valid_1's l2: 0.566455
[350]	training's l2: 0.450145	valid_1's l2: 0.566142
[400]	training's l2: 0.449883	valid_1's l2: 0.565936
[450]	training's l2: 0.449638	valid_1's l2: 0.565838
[500]	training's l2: 0.449398	valid_1's l2: 0.565756
[550]	training's l2: 0.449164	valid_1's l2: 0.565675
[600]	training's l2: 0.448937	valid_1's l2: 0.565613
[650]	training's l2: 0.448718	valid_1's l2: 0.565506
[700]	training's l2: 0.448507	valid_1's l2: 0.565434
[750]	training's l2: 0.448301	valid_1's l2: 0.565451
Early stopping, best iteration is:
[712]	training's l2: 0.448458	valid_1's l2: 0.565419
mean_16_2017: 14109061.89
mean_14_2017: 2736958.14
mean_7_2

/Users/arkarmin/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[50]	training's l2: 0.54421	valid_1's l2: 0.552233
[100]	training's l2: 0.466982	valid_1's l2: 0.474582
[150]	training's l2: 0.45608	valid_1's l2: 0.463594
[200]	training's l2: 0.454199	valid_1's l2: 0.4619
[250]	training's l2: 0.453606	valid_1's l2: 0.461543
[300]	training's l2: 0.453242	valid_1's l2: 0.461429
[350]	training's l2: 0.452952	valid_1's l2: 0.461393
[400]	training's l2: 0.452687	valid_1's l2: 0.461351
[450]	training's l2: 0.452441	valid_1's l2: 0.461326
[500]	training's l2: 0.452195	valid_1's l2: 0.461313
[550]	training's l2: 0.451963	valid_1's l2: 0.461294
[600]	training's l2: 0.451737	valid_1's l2: 0.461281
[650]	training's l2: 0.45153	valid_1's l2: 0.461257
Early stopping, best iteration is:
[649]	training's l2: 0.451535	valid_1's l2: 0.461253
mean_16_2017: 14740615.48
mean_14_2017: 3069550.53
mean_7_2017: 360494.40
mean_3_2017: 121108.33
Validation mse: 0.4620377217114561


In [18]:
print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_train.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('modified_weight.csv', float_format='%.4f', index=None)

Making submission...
